In [2]:
# dependencies
import numpy as np
import pandas as pd

In [3]:
# Loading data
data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

In [4]:
data_matrix_train.shape

(722, 892)

In [5]:
COP_train.shape # KPI, in total 4 KPIs, and there are 722 measurements

(722, 4)

# 3. Least Squares
## Question 3.1

If $Aw = b$ then we have $y(t) = \tilde x(t)^Tw_1+w_0-y(t)\tilde x(t)^Tw_2$. Therefore we have:

$$y(t) = \frac{w_1^T\tilde x(t)+w_0}{\tilde x(t)^Tw_2+1}$$

## Question 3.2

In [6]:
def standardize_data(data):
    mean = np.mean(data, axis=0)
    M = data - mean
    std = np.std(M, axis=0)
    M = M/std
    return M

In [7]:
# standardize the matrix
matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

In [8]:
(-(M.T * COP_train[:,3])).shape

(892, 722)

In [9]:
# constructing A and b for solving min_w ||A w - b||_2**2
## point wise multiplication broadcast
# A of shape (n, 2m+1)
A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T]) # -(M.T * COP_train[:,3]).T shape (722, 892)
b = COP_train[:,3] # KPI number 3, shape (n,1)

In [10]:
# solving the linear question, getting the w vector
# here we have m = d in the definition
w = np.linalg.lstsq(A,b, rcond=None)[0] # of shape 2d+1

## Question 3.3

Evaluate the quality of the solution with MSE and R2

In [11]:
# Constructing matrices for the test set
M_test = (data_matrix_test - matrix_mean) / matrix_std # use training data mean for test, we cannot suppose the results for test
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]

In [12]:
b_predict = A_test @ w
n_test = b_test.shape[0]
mse_1 = 1/n_test * np.sum((b_test - b_predict)**2)
R2_1 = 1- np.sum((b_test - b_predict)**2)/np.sum((b_test - b_test.mean())**2)
mse_1, R2_1

(780.8984793523456, -40.87585582906847)

In [13]:
def get_score(A, w, b):
    residuals = A@w-b
    return (0.5 * np.linalg.norm(residuals)**2)

In [14]:
get_score(A_test, w, b_test)

140952.17552309835

## Question 3.4

We will use the result of the following question with the gradient, and we set the gradient to 0, i.e.: $A^T(Aw-b) + \lambda w = 0$. Therefore solving the problem becomes making $A^TAw + \lambda w = A^Tb$

In [15]:
lambda_ridge = 100
w2 = np.linalg.lstsq(A.T @ A + lambda_ridge * np.identity((A.T @ A).shape[0]),(A.T @ b), rcond=None)[0]

In [16]:
b_predict2 = A_test @ w2
n_test = b_test.shape[0]
mse_2 = 1/n_test * np.sum((b_test - b_predict2)**2)
R2_2 = 1- np.sum((b_test - b_predict2)**2)/np.sum((b_test - b_test.mean())**2)
mse_2, R2_2

(301.0548280945075, -15.14413257455234)

The test MSE is about half of that in the unregularized problem.

## Question 3.5

The gradient of $w\mapsto \frac{1}{2}||Aw-b||^2$ is $ A^T(Ax-b)$, and the gradient of $\frac{\lambda}{2}||w||^2$ is $\lambda w$. Therefore the gradient of the function is 

$$\nabla f_1(w) :\; w\mapsto A^T(Aw-b) + \lambda w$$

To determine the convexity of a function, we have to look into its Hessian matrix.

The Hessian of function $f_1$ is $H(f_1) = A^TA + \lambda I$ which is positive semi definitive. Therefore function $f_1$ is convex.

## Question 3.6

For the gradient descent, we use the process $$x_{k+1} = x_k - \gamma_k \nabla f_1(x_k)$$. 

where $\gamma_k$ is the learning step and here we set it as a constant. We randomly initialize $w$, and the target is to get the optimal $w$.

In [37]:
(A @ w0).shape

(722, 1)

In [38]:
A.T.shape

(1785, 722)

In [46]:
(A.T @ A @ w ).shape

(1785,)

In [45]:
w.shape

(1785,)

In [32]:
b

array([ 8.76491653,  8.8438464 ,  3.76089874,  3.71666913,  3.58255932,
        3.306774  ,  3.30792173,  6.83331686,  3.13708744,  3.33057503,
        3.22320998,  8.12098306,  7.754193  ,  3.29406059,  3.35823478,
        3.52437946,  2.80497745,  4.77146998,  2.73655381,  3.91922239,
        4.03605079,  4.0651159 , 11.10411331,  3.97340026, 11.87813909,
        3.94730954, 10.51994604,  3.84016567,  4.35510147,  4.33740932,
        4.26665541,  4.36421565,  4.37908818, 11.67084938,  4.65837061,
       10.577855  ,  4.32085972, 11.51069335,  4.28548163,  4.74556538,
        4.08024852,  3.26302011,  2.95272548,  9.36877402,  3.96903821,
        4.06816979,  4.1166304 ,  4.26764676,  4.06088352, 10.04707594,
        4.10628677,  9.57462636,  3.72404833,  3.92685151,  9.62867667,
        3.81973149,  4.08658883,  4.11987818,  9.82520075,  9.82459451,
        4.21617925,  8.84144874,  4.66087375,  3.84240524,  7.86076559,
        8.62641723,  3.64344871,  3.81441033,  3.63171803, 10.47

In [47]:
def gradient_f1(A, b, w, lambda_ridge=100):
    return A.T @ A @ w - A.T @ b + lambda_ridge*w
#     return A.T@(A@w-b)+lambda_ridge*w

In [48]:
def descent_f1(A, b, w0, learning_step=0.0001, lambda_ridge=100, stop_condition=1, N_iter_max = 1000):
    w = w0 # initialize the target
    for i in range(N_iter_max):
        gradient = gradient_f1(A,b,w,lambda_ridge)
        if np.linalg.norm(gradient) < stop_condition:
            break # leave the loop if we reach the stop condition
        w = w - learning_step * gradient
    return w, i+1

Here in order to find the Lipschitz, we need to find the maximum value of $||\nabla f_1||$. Involving finding the maximum singular values of $\lambda I + A^TA$. 

In [49]:
lambda_ridge=100
max_eigen = np.linalg.eigvals(A.T @ A).max() + lambda_ridge
L = 1/(max_eigen)
L

(2.863161135661648e-07+0j)

In [54]:
learning_step = 0.9 * 2 * L
d = data_matrix_train.shape[1]
w0 = np.zeros((2*d+1,))
w_gradient_descent, N_iter_final = descent_f1(A, b, w0,learning_step=learning_step, N_iter_max = 100000)
w_gradient_descent, N_iter_final

(array([-0.01238055+0.j,  0.05775865+0.j, -0.00111773+0.j, ...,
         0.01579911+0.j, -0.03571617+0.j,  0.01335282+0.j]),
 54192)

In [23]:
# w_gradient_descent.shape

(1785, 722)

# Regularization for a sparse model

## Question 4.1

Here we can write $f_2 = \min_w\frac{1}{2}||Aw-b||^2$ and $g_2 = \lambda ||w||_1$. Where $f_2$ is differentiable and $g_2$ is convex lower-semicontinous. The gardient of $f_2$ is $$\nabla f_2(w) :\; w\mapsto A^T(Aw-b)$$

The proximal operator of $g_2$ is $\mathrm{prox}_g(x) = \arg\min_yg(y) + \frac{1}{2}||x-y||^2$.

$$
\mathrm{prox}_g(x) =
\begin{cases}
x - \lambda, & \text{if } x > \lambda  \\
0, & \text{if } |x| \leq \lambda  \\
x + \lambda, & \text{if } x < -\lambda
\end{cases}
$$


## Question 4.2

In [55]:
def prox_g(x, lambda_lasso = 200):
    res = []
    for i in x:
        if i > lambda_lasso:
            res.append(i-lambda_lasso)
        else if i < -lambda_lasso:
            res.append(i+lambda_lasso)
        else:
            res.append(0)
    return np.array(res)

SyntaxError: expected ':' (84707397.py, line 6)

In [ ]:
def gradient_f2(A, b, w):
    return A.T@(A@w-b)

In [ ]:
def descent_prox_lasso(A, b, w0, learning_step, lambda_lasso = 200, stop_condition=1, N_iter_max = 1000):
    w = w0 # initialize the target
    for i in range(N_iter_max):
        gradient = gradient_f2(A,b,w,lambda_ridge)
        if np.linalg.norm(gradient) < stop_condition:
            break # leave the loop if we reach the stop condition
        w = prox_g(w - learning_step * gradient_f2(A,b,w), lambda_lasso = lambda_lasso * learning_step)
    return w, i+1